# Вебинар 8. Консультация

---

## 1. Recap по финальному проекту

**Основное**
- Дедлайн - 26 июля 23:59
- Целевая метрика precision@5 на retai_test1.csv
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

In [2]:
# linear = w1*target1 + w2*target2 + w_n * target_n

---

## Разбор multiple-target

## 2. Ответы на вопросы

Как делать кросс-валидацию?
- [Статья](https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)
- Смотреть на метрику на отложенной выборке, меняя гиперпараметры

Какие гиперпараметры оптимизировать у ALS:
- Кол-во факторов 
- regularization / iterations

Как можно делать кросс-валидацию (отложенная выборка)

In [2]:
from recommenders import MainRecommender

my_model = MainRecommender(data)
user_item_matrix = my_model.user_item_matrix

for factors in [20, 50, 100]:
    my_model.fit(user_item_matrix, n_factors=factors, regularization=0.001, iterations=15, num_threads=4)
    # calculate precision@5 on validation

In [ ]:
# your_questions

---

## 3. Разбор соревнования по рекомендациям топ-30 товаров X5 Retail Hero

Постановка задачи + особенности данных + подход к решению из [презентации](https://vk.com/away.php?utf=1&to=https%3A%2F%2Fgithub.com%2Faprotopopov%2Fretailhero_recommender%2Fblob%2Fmaster%2Fslides%2Fretailhero_recommender.pdf) решения 1-ого места

Бонус - [код](https://github.com/aprotopopov/retailhero_recommender) решения 1-ого места

## Основные подходы участников из топ-15

### Генерация кандидатов

В основном генерировали k = {50, 100} кандидатов, чтобы модель 2-ого уровня отрабатывала за разумное время

- Прошлые покупки юзера + топ-популярных, если покупок < k --> recall@100 ~ 42%
- Из ALS, item2item моделей
- implicit.nn.CosineRecommender

### Фичи в модели 2-ого уровня
Коллаборативные:
- biases + embeddings из коллаборативной фильтрации / Скоры item2item моделей
- TF-IDF матирца item-ов c > N покупками
- TF-IDF + TSNE/UMAP

handcrafted фичи для товаров:
- категории товаров
- нормированная частота покупки товара для каждого клиента
- кол-во магазинов, в которых продавался товар
- кол-во транзакций клиента
- mean / max / std кол-ва уникальных товаров в корзине клиента 
- mean / max / std кол-ва уникальных категорий в корзине клиента 

handcrafted фичи для юзеров:
- Средний чек
- Средняя цена одного купленного товара
- Среднее кол-во дней между покупками / с последней покупки    
- Количество уникальных покупок по всем категориям transaction_id, product_id, store_id, level_i_id
- Признаки с накопленными бонусами
- Средняя скидка, доля купленных товаров со скидками

Интересные:
- Факт заказа каждого товара в последних 5 транзакциях в виде последовательности бит (категориальная фича). 
10001 - купил товар в последней транзакции и 5 транзакий назад (feature hashing)
- item co_ocurrence
- word2vec эмбеддинги товаров (альтернативное название - item2vec, prod2vec)
- Расстояние от word2vec эмбеддинга товара до среднего эмбедиингов товаров, купленных юзером

In [ ]:
# Работа с эмбеддингами товаров на примере ALS вместо word2vec
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares()
model.fit(...)

model.item_factors  # Добавить как фичи в модель 2-ого уровня

# Предположение: эмбеддинг юзера = среднее эмбеддингов купленниых им товаров
# эмбеддинг товара - эмбеддинг юзера  # чем разница меньше, тем товар ближе к интересам пользователя

### Модели 2-ого уровня
- классификация через LightGBM
- ранжирование CatBoost через YetiRank, YetiRankPairwise